In [1]:
##
import sys
import os
import pickle
import sys
import warnings
import enum
from enum import Enum
from typing import Optional, List, Dict, Callable

##
import matplotlib
from pathlib import Path

##
sys.path.append("./src/")
sys.path.append("./")
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

##
from finrl import config as finrl_config
from finrl.main import check_and_make_directories
from finrl.config_tickers import DOW_30_TICKER
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
from finrl.config import (
    TEST_END_DATE,
    TRAINED_MODEL_DIR,
)
from finrl.meta.preprocessor.preprocessors import data_split
from finrl.plot import backtest_stats, convert_daily_return_to_pyfolio_ts, get_baseline

##
import yfinance as yf

##
from meta.data_processors.yahoofinance import Yahoofinance

/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [2]:
def config():
    # #
    warnings.filterwarnings("ignore", category=UserWarning)  # TODO: zipline problem
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    # os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # INFO, WARN are ignored and only ERROR messages will be printed

    # #
    matplotlib.use("Agg")

    # #
    check_and_make_directories(
        [
            finrl_config.DATA_SAVE_DIR,
            finrl_config.TRAINED_MODEL_DIR,
            finrl_config.TENSORBOARD_LOG_DIR,
            finrl_config.RESULTS_DIR,
        ]
    )


config()

In [3]:
##
import pandas as pd
import tqdm
from dotenv import load_dotenv
import financedatabase as fd
import fundamentalanalysis as fa

##
from common.utils import now_time
from configuration.settings import ProjectDir
from rl.data.Company import Company

In [4]:
prj_dir = ProjectDir(root=Path("/Users/zlapik/my-drive-zlapik/0-todo/ai-investing"))
load_dotenv(prj_dir.root.joinpath("env/.env"))

_TRAIN_DATA_START = "2010-01-01"
_TRAIN_DATA_END = "2021-12-31"
_TEST_DATA_START = "2021-01-01"
_TEST_DATA_END = "2021-12-31"
tickers = DOW_30_TICKER

In [5]:
try:
    print(os.getenv("EOD_HISTORICAL_DATA_API"))
    print(os.getenv("NASDAQ_HISTORICAL_DATA_API"))
    print(os.getenv("ALPHA_VANTAGE_API"))
    print(os.getenv("FINANCIAL_MODELING_PREP_API"))
except:
    raise ValueError("ERROR: API keys")

639b4cc5af44e7.93519913
B9p3RKb-K6xWwBRLso7Z
G21Y7NB8U8VJFFIH
6b0006554aa86e3e21493713af2af3c4



## Financial Database


In [6]:
# Obtain all countries from the database
equities_countries = fd.show_options("equities", "countries")

# Obtain all sectors from the database
equities_sectors = fd.show_options("equities", "sectors")

# Obtain all industries from the database
equities_industries = fd.show_options("equities", "industries")

# Obtain all countries + sectors + industries from the database
equities_all_categories = fd.show_options("equities")

In [7]:
print(equities_countries)
print()
print(equities_sectors)
print()
print(equities_industries)
print()
print(equities_all_categories)

['Afghanistan', 'Anguilla', 'Argentina', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bangladesh', 'Barbados', 'Belgium', 'Belize', 'Bermuda', 'Botswana', 'Brazil', 'British Virgin Islands', 'Cambodia', 'Canada', 'Cayman Islands', 'Chile', 'China', 'Colombia', 'Costa Rica', 'Cyprus', 'Czech Republic', 'Denmark', 'Dominican Republic', 'Egypt', 'Estonia', 'Falkland Islands', 'Finland', 'France', 'French Guiana', 'Gabon', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Greenland', 'Guernsey', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Ivory Coast', 'Japan', 'Jersey', 'Jordan', 'Kazakhstan', 'Kyrgyzstan', 'Latvia', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macau', 'Macedonia', 'Malaysia', 'Malta', 'Mauritius', 'Mexico', 'Monaco', 'Mongolia', 'Montenegro', 'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Netherlands', 'Netherlands Antilles', 'New Zealand', 'Nigeria', 'Norway', 'Panama', 'Papua New Guinea', 'Peru', 'Ph

## Financial Analyses

In [8]:
ticker = "MSFT"
api_key = os.getenv("FINANCIAL_MODELING_PREP_API")

In [9]:
# Show the available companies
companies = fa.available_companies(api_key)

In [10]:
# # Collect general company information
# profile = fa.profile(ticker, api_key)
#
# # Collect recent company quotes
# quotes = fa.quote(ticker, api_key)
#
# # Collect market cap and enterprise value
# entreprise_value = fa.enterprise(ticker, api_key)
#
# # Show recommendations of Analysts
# ratings = fa.rating(ticker, api_key)
#
# # Obtain DCFs over time
# dcf_annually = fa.discounted_cash_flow(ticker, api_key, period="annual")
#
# # Collect the Balance Sheet statements
# balance_sheet_annually = fa.balance_sheet_statement(ticker, api_key, period="annual")
#
# # Collect the Income Statements
# income_statement_annually = fa.income_statement(ticker, api_key, period="annual")
#
# # Collect the Cash Flow Statements
# cash_flow_statement_annually = fa.cash_flow_statement(ticker, api_key, period="annual")
#
# # Show Key Metrics
# key_metrics_annually = fa.key_metrics(ticker, api_key, period="annual")
#
# # Show a large set of in-depth ratios
# financial_ratios_annually = fa.financial_ratios(ticker, api_key, period="annual")
#
# # Show the growth of the company
# growth_annually = fa.financial_statement_growth(ticker, api_key, period="annual")
#
# # Download general stock data
# stock_data = fa.stock_data(ticker, period="ytd", interval="1d")
#
# # Download detailed stock data
# stock_data_detailed = fa.stock_data_detailed(ticker, api_key, begin="2000-01-01", end="2020-01-01")
#
# # Download dividend history
# dividends = fa.stock_dividend(ticker, api_key, begin="2000-01-01", end="2020-01-01")

In [11]:
# # Obtain DCFs over time
# dcf_quarterly = fa.discounted_cash_flow(ticker, api_key, period="quarter")
#
# # Collect the Balance Sheet statements
# balance_sheet_quarterly = fa.balance_sheet_statement(ticker, api_key, period="quarter")
#
# # Collect the Income Statements
# income_statement_quarterly = fa.income_statement(ticker, api_key, period="quarter")
#
# # Collect the Cash Flow Statements
# cash_flow_statement_quarterly = fa.cash_flow_statement(ticker, api_key, period="quarter")
#
# # Show Key Metrics
# key_metrics_quarterly = fa.key_metrics(ticker, api_key, period="quarter")
#
# # Show a large set of in-depth ratios
# financial_ratios_quarterly = fa.financial_ratios(ticker, api_key, period="quarter")
#
# # Show the growth of the company
# growth_quarterly = fa.financial_statement_growth(ticker, api_key, period="quarter")

In [13]:
from rl.data.financial_modeling_prep import download
import concurrent

all_symbols_info = dict()

end = 1000
step = end // 10
with concurrent.futures.ProcessPoolExecutor() as executor:
    result = [executor.submit(download, companies.index[i : i + step - 1], api_key) for i in range(0, end, step)]

    for i in concurrent.futures.as_completed(result):
        try:
            all_symbols_info.update(i.result())
        except Exception as e:
            print(e)

Symbol: RY: 100%|██████████| 99/99 [07:04<00:00,  4.29s/it]


In [15]:
all_symbols_info.__len__()

990

In [16]:
for k, v in all_symbols_info.items():
    print(k)

BMN
SC0I.DE
NGMDF
RSCZF
GC1.AX
DBV
BRG-B.ST
ATO.PA
JHMH
B500.DE
72HN.L
RRIF
BRO.V
DLTA
FORZ.JK
TIPE.MI
VNET.L
HLLGY
WLMTF
UB32.L
TSYHF
GBCHF
GFOF
QUASX
IAFMX
CIIGW
LICH
MZPS
CYLYF
CTM.V
ROVMF
RUSE.L
FZRO
CEVE
004870.KS
ZEN.V
KARW.JK
ESUG.MI
PXTIX
BITF
XMRC.L
EQH
9405.T
TSCIX
GACIX
WTSHF
EUN6.DE
FOOD.SW
OPCGF
STYAX
IE00BDDRF924.SG
WRGL
0K2F.L
001318.SZ
ADORWELD.NS
JRUB.DE
18M0.DE
FTBYF
ZTNO
FRVLX
1896.HK
UB23.L
CDUTF
005750.KS
PEZ
AONNF
EFX.TO
ACRV
SYRSW
PTE.V
GAAEX
NPTH
GBSPX
WDBG
EMRG.CN
ZVLO
AAEEF
CDNIF
CWLDY
BOGIX
GBCEY
ZNWLF
XCOMQ
3762.T
8487.HK
VHAQ-WT
JOBY-WT
CMX1.L
DNMDX
VITNX
FTXR
VSCSX
2360.TW
GSDC
BDFIX
BRLIW
STRI
MAQCW
TTDAX
JDC.DE
DOSE.CN
ALCOR.PA
IIXIX
USX1.DE
RVLCF
HUGS-WT
PRJPX
WNRC
FIGTX
JWSM-WT
BARU.V
SHPW-WT
GWA.V
LBNK.V
SINGY
BSKYW
AME.V
MOON.V
AKEL-D.ST
HTHIF
FLDFX
GCEQX
USRTX
DEVM
1600.HK
DCSX
DSGT
HMCX.L
300468.SZ
1848.HK
QDVA.DE
CTTH
PQTAX
MGUY
VMGRX
IBXS
SCFR
BNKR.CN
VALSX
88Q.DE
ACIIW
LI
JPTS.L
VUKG.L
TNBI
V50D.DE
TGRP
IAEX.AS
NLPXF
JPEU.AS
DENKF
ABXX.NE
HLTH.L

In [17]:
filename = prj_dir.dataset.financial_modeling_prep.joinpath(f"all_companies_info_till_{now_time()}.pkl")
filename

PosixPath('/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/dataset/financialmodelingprep/all_companies_info_till_2022-12-16T01-25-41.pkl')

In [18]:
# Save
with open(file=filename, mode="wb") as f:
    pickle.dump(all_symbols_info, f)

In [47]:
# Load
with open(filename, mode="rb") as f:
    all_companies: Dict[str, Company] = pickle.load(f)

sys.getsizeof(all_companies)

360

In [48]:
for k, v in all_companies.items():
    print(k)

ULST
AVUV
287310.KS
EMV.L
JRDM.L
IDVY.L
1944.T
CHAM
GCBLX
FSA.AX
